# Walkthrough Challenge 4 - Chat with your own data

Duration: 30 minutes

## Overview
- In this challenge, you will learn on how to use the `Chat` api to keep history of the conversation within a chatbot.
- You will also learn to implement the *Retrieval Augmented Generation* (RAG) architecture to chat with your own data.

## Prerequisites

- Please ensure that you have completed the [Setup](../setup/setup.ipynb) before starting this challenge.

### Task 1: Configure and Initialize Semantic Kernel

⚠️ Note: You should have already completed all tasks on the [Setup](../setup/setup.ipynb). If you have not, please go back and complete it now.

#### Step 1: Load Semantic Kernel settings

In this step, we will load the Semantic Kernel settings that we created in the [Setup](../setup/setup.ipynb) notebook.

In [21]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.13.0-preview"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, 1.13.0-preview"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.13.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.Sqlite, 1.13.0-alpha"
#r "nuget: Microsoft.KernelMemory.Core, 0.61.240524.1"
#r "nuget: Microsoft.KernelMemory.SemanticKernelPlugin, 0.61.240524.1"

#!import ../setup/config/Settings.cs
#!import ../setup/config/Utils.cs

Installed Packages Microsoft.KernelMemory.Core, 0.61.240524.1 Microsoft.KernelMemory.SemanticKernelPlugin, 0.61.240524.1 Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Connectors.Sqlite, 1.13.0-alpha Microsoft.SemanticKernel.Planners.Handlebars, 1.13.0-preview Microsoft.SemanticKernel.Planners.OpenAI, 1.13.0-preview Microsoft.SemanticKernel.Plugins.Memory, 1.13.0-alpha

#### Step 2: Initialize Semantic Kernel

In [22]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.Sqlite;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.KernelMemory;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.DotNet.Interactive;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile("../setup/config/settings.json");

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

### Task 2: Create a chatbot like experience, mainting the context of the conversation 

We will start by creating a `ChatHistory` object to maintain the context of the conversation. This object will be used to store the conversation history, namely the system prompts, user inputs and the assistant responses.

The `ChatHistory` object will be initialized with the system prompt.`

In [23]:
var chatHistory = new ChatHistory("You are an historian, expert in the Seven Wonders of the Ancient World. Be concise and informative.");

The `MessageOutputAsync` method will allow us to print the latest message from the chat history.

In [24]:
await Utils.MessageOutputAsync(chatHistory);

system: You are an historian, expert in the Seven Wonders of the Ancient World. Be concise and informative.
------------------------


To interact with the chatbot, we will require the `ChatCompletionService`. This will be used to interact with the Large Language Model to generate responses to the user inputs.

After adding the user input to the chat history, we will use the `ChatCompletionService` to generate a response. This response will be added to the chat history and printed to the user.

In [25]:
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

var executionSettings = new OpenAIPromptExecutionSettings
{
    MaxTokens = 500,
    Temperature = 0.2,
    TopP = 0.5
};

chatHistory.AddUserMessage("Hi, what is the Great Pyramid of Giza?");
await Utils.MessageOutputAsync(chatHistory);

var reply = await chatCompletionService.GetChatMessageContentAsync(
        chatHistory,
        executionSettings: executionSettings,
        kernel: kernel);

chatHistory.Add(reply);
await Utils.MessageOutputAsync(chatHistory);

user: Hi, what is the Great Pyramid of Giza?
------------------------
assistant: The Great Pyramid of Giza, located in Egypt, is the oldest and only surviving monument of the Seven Wonders of the Ancient World. It was constructed around 2580–2560 BC during the reign of Pharaoh Khufu (also known as Cheops). Originally standing at 146.6 meters (481 feet), it was the tallest man-made structure in the world for over 3,800 years. The pyramid served as a tomb and was built using millions of limestone blocks, with some weighing as much as 15 tons. It is part of a larger complex that includes two other major pyramids, smaller pyramids, temples, and the Great Sphinx.
------------------------


Let's add the new message to chat history. You will see that the chat model will use the context of the conversation to generate a response.

Take note that chat history counts towards your token usage, so be mindful of the number of messages you send. Strategies such as counting the number of tokens and truncating the history to only include the last few messages can be used to manage token usage.

In [26]:
chatHistory.AddUserMessage("And do you know who built it?");
await Utils.MessageOutputAsync(chatHistory);

var reply = await chatCompletionService.GetChatMessageContentAsync(
        chatHistory,
        executionSettings: executionSettings,
        kernel: kernel);

chatHistory.Add(reply);
await Utils.MessageOutputAsync(chatHistory);

user: And do you know who built it?
------------------------
assistant: The Great Pyramid of Giza was commissioned by Pharaoh Khufu of the Fourth Dynasty of ancient Egypt. The construction is believed to have been overseen by his vizier, Hemiunu. Contrary to popular belief, the pyramid was not built by slaves but rather by a large workforce of skilled laborers, including stone masons, engineers, and workers, who were likely conscripted for labor as a form of tax payment and were well-fed and housed. Recent discoveries suggest that thousands of laborers worked in shifts to construct the pyramid over a period of approximately 20 years.
------------------------


You can experiment interacting with the chatbot by adding more messages to the chat history and observing the responses.

In [27]:
do
{
    try
    {
        var ask = await InteractiveKernel.GetInputAsync("Ask a question to the assistant: ");
        chatHistory.AddUserMessage(ask);
        await Utils.MessageOutputAsync(chatHistory);

        var reply = await chatCompletionService.GetChatMessageContentAsync(
                chatHistory,
                executionSettings: executionSettings,
                kernel: kernel);

        chatHistory.Add(reply);
        await Utils.MessageOutputAsync(chatHistory);
    }
    catch (Exception)
    {
        // break the loop if the user cancels the input
        break;
    }
} while (true);


Error: Input request cancelled

### Task 3: Use your own data to chat with the model

In some scenarions, you may want the model to respond with specific information based on your own data. For example, you may have internal documentation about an appliance, and the model should respond with information that is specific to that appliance, not from the general knowledge it has been trained on.

This approach is called *Retrieval Augmented Generation* (RAG). In this approach, the model first retrieves relevant information from a knowledge source, and then generates a response based on the retrieved information.

In [28]:
var memorybuilder = new KernelMemoryBuilder();

memorybuilder.WithAzureOpenAITextEmbeddingGeneration(new AzureOpenAIConfig()
{
    Auth = AzureOpenAIConfig.AuthTypes.APIKey,
    APIKey = apiKey,
    APIType = AzureOpenAIConfig.APITypes.EmbeddingGeneration,
    Endpoint = azureEndpoint,
    Deployment = "Text-embedding-ada-002"
});

memorybuilder.WithAzureOpenAITextGeneration(new AzureOpenAIConfig()
{
    Auth = AzureOpenAIConfig.AuthTypes.APIKey,
    APIKey = apiKey,
    APIType = AzureOpenAIConfig.APITypes.ChatCompletion,
    Endpoint = azureEndpoint,
    Deployment = model
});

var memory = memorybuilder.Build();


Let's start by importing a document about Germany Labour Law using the *Semantic Memory* library.

In [29]:

await memory.ImportWebPageAsync(
    "https://web.archive.org/web/20231219085537/https://www.ilo.org/ifpdial/information-resources/national-labour-law-profiles/WCMS_158899/lang--en/index.htm");


After that, we can import the *Semantic Memory* library and use it to add the document to the knowledge base. We will do it via the `MemoryPlugin` class.

In [30]:
using Microsoft.KernelMemory;

var memoryPlugin = kernel.ImportPluginFromObject(new MemoryPlugin(memory));

Let's create a prompt representig the Retrieval Augmented Generation (RAG) approach. The prompt will have an input variable that will represent the user question and will have a function call to the `MemoryPlugin` to retrieve the relevant information from the knowledge base, using the `ask` function.

In [31]:
var skPrompt = """
Question to Memory: {{$input}}

Answer from Memory: {{MemoryPlugin.Ask $input}}

If the answer is empty say 'I don't know' otherwise reply with a preview of the answer,
truncated to 15 words. Prefix with one emoji relevant to the content.
""";

var ragFunction = kernel.CreateFunctionFromPrompt(skPrompt);

And we're ready to use the RAG approach to chat with the model. Let's ask a question about Germany Labour Law and see how the model responds.

In [34]:
var answer = await ragFunction.InvokeAsync(kernel, "How many vacations days do I get in Germany?");

Console.WriteLine(answer);

🏝 In Germany, the statutory minimum leave entitlement is 24 days per calendar year, not counting...


And to test the case when the model does not have the information, let's ask a question about Portugal Labour Law.

In [33]:
var answer = await ragFunction.InvokeAsync(kernel, "How many vacations days do I get in Portugal?");

Console.WriteLine(answer);

🌴 I don't know


You successfully completed challenge 4! 🚀🚀🚀

 **[Home](../../Readme.md)** - [Next Challenge Solution](../challenge-5/solution.ipynb)